# 準備

In [1]:
!pip install gensim
!pip install SudachiPy==0.5.4 ginza==4.0.6 ja-ginza==4.0.0 
!pip install pytorch_lightning
# !pip install emoji
!pip install transformers[ja]
!pip install sentencepiece

!pip install fugashi
!pip install ipadic

!pip install deep_translator
!pip install googletrans==4.0.0-rc1

You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.


# import

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os
import re
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

#one_hotベクトル
from sklearn.feature_extraction.text import TfidfVectorizer

#ベクトル化
from gensim.models import word2vec

#データ分割用
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import sklearn

import torch

from transformers import AutoTokenizer
from transformers import BertJapaneseTokenizer, AutoModelForMaskedLM, BertModel

# 翻訳
# -*- coding: utf-8
from googletrans import Translator

# データの読み込み

In [3]:
files = glob("*.csv")
files

['FM4002H.csv', '20230203081922.csv']

In [4]:
vocal_xlsx = files[1]
tqnet_xlsx = files[0]

_vocal_df = pd.read_csv(vocal_xlsx, skiprows=3)
_tqnet_df = pd.read_csv(tqnet_xlsx)

In [5]:
vocal_df = _vocal_df[["申出内容","車名"]]
vocal_df.columns = ["text","car_name"]
vocal_df.head()

,text,car_name
0,ドライブレコーダーを目立たなくしてほしい\n前後録画タイプのドライブレコーダーを取り付けたい...,シエンタ
1,３列目が窮屈で居心地が悪い\n３列目の背もたれの高さが不十分で足元スペースも余裕がない。先代...,シエンタ
2,アルミホイールの設定が欲しかった。\nシエンタは全車スチールホイールなので、他車種のようにア...,シエンタ
3,用品のサイドエンブレムを取り付けたがバックドアの車名にも同じようなエンブレムを取り付けたい\...,シエンタ
4,1列目のドアポケットの位置を持ちやすい高さにして欲しい\n1列目のドアから乗り降りをしようと...,シエンタ


In [6]:
tqnet_df = _tqnet_df[["要約ｺﾒﾝﾄ","市情報No"]]
tqnet_df.columns = ["text","no"]
tqnet_df.head()

,text,no
0,走行中凸凹路時、ブレーキ時に異音(ガキン/ゴキン)→ロアアーム×メンバー間に石→石除去,20230104-14248-0
1,販売店指摘）ﾒｰﾀｰﾊﾟﾈﾙに傷がある→製造不良→ｲﾝｽﾄﾙﾒﾝﾄｸﾗｽﾀﾌｨﾆｯｼｭﾊﾟﾈﾙ交換,20230104-52107-0
2,販売店指摘）ｾｶﾝﾄﾞｼｰﾄ跳ね上げ出来ない→修理無し。,20230104-62922-0
3,【第2報】ﾏﾙﾁｲﾝﾌｫﾒｰｼｮﾝﾃﾞｨｽﾌﾟﾚｲ画面が黒画点滅。→ｺﾝﾋﾞﾈｰｼｮﾝ ﾒｰ...,20230104-A2217-0
4,(販売店)ﾌﾛﾝﾄﾄﾞｱﾄﾘﾑﾎﾞｰﾄﾞ上部手すりについて,20230104-Z0544-0


# データクリーニング
## vocal (X)

In [7]:
vocal_df["text"] = vocal_df["text"].str.replace("\n", ' ')
vocal_df["text"] = vocal_df["text"].str.replace(re.compile("（\w+）"), '')
vocal_df["text"] = vocal_df["text"].str.replace(re.compile("[。、]"), ' ')
vocal_df = vocal_df.iloc[0:100, :]
vocal_df.head()

/tmp/ipykernel_249/84199848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocal_df["text"] = vocal_df["text"].str.replace("\n", ' ')
/tmp/ipykernel_249/84199848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocal_df["text"] = vocal_df["text"].str.replace(re.compile("（\w+）"), '')
/tmp/ipykernel_249/84199848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,text,car_name
0,ドライブレコーダーを目立たなくしてほしい 前後録画タイプのドライブレコーダーを取り付けたいが...,シエンタ
1,３列目が窮屈で居心地が悪い ３列目の背もたれの高さが不十分で足元スペースも余裕がない 先代の...,シエンタ
2,アルミホイールの設定が欲しかった シエンタは全車スチールホイールなので 他車種のようにアル...,シエンタ
3,用品のサイドエンブレムを取り付けたがバックドアの車名にも同じようなエンブレムを取り付けたい ...,シエンタ
4,1列目のドアポケットの位置を持ちやすい高さにして欲しい 1列目のドアから乗り降りをしようとす...,シエンタ


## tqnet (Y)

In [8]:
tqnet_df["text"] = tqnet_df["text"]\
    .str.replace("\n", ' ')\
    .str.replace("×", 'と')\
    .str.replace("&", 'と')\
    .str.replace("。", '')\
    .str.replace("→", ' ')\
    .str.replace("�", '')\
    .str.replace("販売店指摘）", '')\
    .str.replace(re.compile("（\w+）"), "")\
    .str.replace(re.compile("【第\w+報】"), '')

tqnet_df = tqnet_df.iloc[0:100, :]
tqnet_df.head(20)

/tmp/ipykernel_249/3662075805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tqnet_df["text"] = tqnet_df["text"]\


,text,no
0,走行中凸凹路時、ブレーキ時に異音(ガキン/ゴキン) ロアアームとメンバー間に石 石除去,20230104-14248-0
1,ﾒｰﾀｰﾊﾟﾈﾙに傷がある 製造不良 ｲﾝｽﾄﾙﾒﾝﾄｸﾗｽﾀﾌｨﾆｯｼｭﾊﾟﾈﾙ交換,20230104-52107-0
2,ｾｶﾝﾄﾞｼｰﾄ跳ね上げ出来ない 修理無し,20230104-62922-0
3,ﾏﾙﾁｲﾝﾌｫﾒｰｼｮﾝﾃﾞｨｽﾌﾟﾚｲ画面が黒画点滅 ｺﾝﾋﾞﾈｰｼｮﾝ ﾒｰﾀ交換,20230104-A2217-0
4,(販売店)ﾌﾛﾝﾄﾄﾞｱﾄﾘﾑﾎﾞｰﾄﾞ上部手すりについて,20230104-Z0544-0
5,ｵｰﾃﾞｨｵ画面が真っ黒で表示されない ﾗｼﾞｵとﾃﾞｨｽﾌﾟﾚｲﾚｼｰﾊﾞ交換,20230105-14332-0
6,走行中に故障ﾒｯｾｰｼﾞ ﾀﾞｲｱｸﾞB249300(右AHSﾄﾞﾗｲﾊﾞ異常)入力 ﾍｯﾄ...,20230105-25515-0
7,冷却水が確認しづらい,20230105-81150-0
8,運転席側ｱｯﾊﾟｰﾎﾞｯｸｽを閉めてもﾛｯｸがかからない ﾛｯｸ部摺動不良 ｲﾝｽﾄﾙﾒﾝﾄ...,20230105-94518-0
9,左ｽﾗｲﾄﾞﾄﾞｱがｱｳﾄｻｲﾄﾞﾊﾝﾄﾞﾙで開かない ｲﾝｻｲﾄﾞﾊﾝﾄﾞﾙAssy推定交換,20230105-94851-0


# テキストベクトル化(辞書なし検討) - Tokenizer活用
## X_label creation (768次元)

In [9]:
# tokenizerとmodelのload
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model = model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 日本語を英語に変換

In [10]:
d_t = []
d_v = []
translator = Translator()
for t_data in tqnet_df["text"]:
    translated_t = translator.translate(t_data)
    d_t.append({
        "text": translated_t.text
    })
new_tqnet_df = pd.DataFrame(d_t)

for v_data in vocal_df["text"]:
    translated_v = translator.translate(v_data)
    d_v.append({
        "text": translated_v.text
    })
new_vocal_df = pd.DataFrame(d_v)

In [11]:
new_vocal_df.head()

,text
0,I want the drive recorder to be less noticeabl...
1,"The 3rd row is cramped and uncomfortable, the ..."
2,I wanted an aluminum wheel setting Sienta was ...
3,"I attached the side emblem of the supplies, bu..."
4,It is easy to have the position of the first -...


In [12]:
new_tqnet_df.head()

,text
0,Stone removal between uneven roads during unev...
1,Produced defective instle cluster pans replace...
2,No repair that can not be jumped up
3,Directi -in -site Design screen is replaced by...
4,(Dealer) About the upper handrail of the Futt ...


In [13]:
#各データの形式を整える
max_length = 256

x_dataset = []
# labels = []
for i in range(len(vocal_df)):
    # 記事から文章を抜き出し符号化を行う
    lines = new_vocal_df.iloc[i,0].splitlines()
    text = '\n'.join(lines)
    encoding = tokenizer(
        text,
        max_length = max_length,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt'
        )
    encoding = {k: v.cuda() for k, v in encoding.items()}
    attention_mask = encoding['attention_mask']
    
    #文章ベクトルを計算
    with torch.no_grad():
        output = model(**encoding)
        last_hidden_state = output.last_hidden_state
        averaged_hidden_state =(last_hidden_state*attention_mask.unsqueeze(-1)).sum(1)/attention_mask.sum(1,keepdim=True) 
        
    #文章ベクトルとラベルを追加
    x_dataset.append(averaged_hidden_state[0].cpu().numpy())
    # label = vocal_df.iloc[i,1]
    # labels.append(label)

#ベクトルとラベルをnumpy.ndarrayにする
x_dataset = np.vstack(x_dataset)
# labels = np.array(labels)

## Y_label creation

In [14]:
max_length = 256

y_dataset = []
for i in range(len(tqnet_df)):
    # 記事から文章を抜き出し符号化を行う
    lines = new_tqnet_df.iloc[i,0].splitlines()
    text = '\n'.join(lines)
    encoding = tokenizer(
        text,
        max_length = max_length,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt'
        )
    encoding = {k: v.cuda() for k, v in encoding.items()}
    attention_mask = encoding['attention_mask']
    
    #文章ベクトルを計算
    with torch.no_grad():
        output = model(**encoding)
        last_hidden_state = output.last_hidden_state
        averaged_hidden_state =(last_hidden_state*attention_mask.unsqueeze(-1)).sum(1)/attention_mask.sum(1,keepdim=True) 
        
    #文章ベクトルとラベルを追加
    y_dataset.append(averaged_hidden_state[0].cpu().numpy())

#ベクトルとラベルをnumpy.ndarrayにする
y_dataset = np.vstack(y_dataset)

# テキストのベクトル化(model)

In [15]:
print(x_dataset.shape)
print(y_dataset.shape)

(100, 768)
(100, 768)


In [16]:
import sklearn
data = sklearn.metrics.pairwise.cosine_similarity(x_dataset,y_dataset)

In [17]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.797882,0.746289,0.770436,0.813437,0.792608,0.807419,0.780044,0.787922,0.844097,0.824532,...,0.804347,0.771259,0.823457,0.775656,0.796135,0.734733,0.798155,0.802129,0.808284,0.759767
1,0.829850,0.786384,0.808748,0.815052,0.834886,0.803724,0.794861,0.797472,0.856149,0.857215,...,0.851889,0.779995,0.846684,0.811521,0.787790,0.785515,0.778649,0.845982,0.797243,0.805645
2,0.784139,0.768981,0.791219,0.788124,0.793354,0.808465,0.760793,0.774583,0.828110,0.822474,...,0.812475,0.769597,0.817883,0.777364,0.776881,0.732302,0.789233,0.811599,0.799613,0.780508
3,0.822109,0.791857,0.803648,0.831864,0.837648,0.835388,0.821087,0.776869,0.844249,0.871222,...,0.842058,0.816799,0.856706,0.792071,0.823120,0.789165,0.815180,0.857404,0.804298,0.818871
4,0.802178,0.737514,0.781513,0.798176,0.809763,0.797998,0.753952,0.781970,0.858167,0.849119,...,0.848776,0.758851,0.809833,0.788486,0.770415,0.755648,0.771565,0.802834,0.789255,0.756543


## headerの作成

In [18]:
t_labels = [label for label in tqnet_df["text"]]
df.columns = t_labels

v_labels = [label for label in vocal_df["text"]]
df["Vocal/TQ-NET"] = v_labels

In [19]:
first_column = df.pop("Vocal/TQ-NET")
df.insert(0,"Vocal / TQ-NET",first_column)

In [22]:
df = df.style.background_gradient()

AttributeError: 'Styler' object has no attribute 'style'

In [21]:
df.to_excel("類似データ.xlsx", index=False)

In [23]:
df